In [1]:
import numpy as np
import cv2
import subprocess
import torch
from PIL import Image
import os
from scipy.stats import hmean
import sys
import matplotlib.pyplot as plt

# 아래부터 추가해야 하는 라이브러리
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import VGG

In [9]:
location = "UD" # 불러올 이미지들의 디렉토리 명
road_real_distance = 300 # 도로의 폭은 300cm 로 지정 / 만일 다른 기준객체를 참고하려면 해당 기준객체의 실 cm 값으로 변경

s = 4 # 침수 전 이미지 프레임의 순서 
f = 3 # 침수 후 이미지 프레임의 순서

# image frame 불러오기
img_1 = cv2.imread(f'example/{location}/{s}.jpg', cv2.IMREAD_COLOR) # 물 웅덩이 box 칠, 첫번째 이미지
img_n = cv2.imread(f'example/{location}/{f}.jpg', cv2.IMREAD_COLOR) # n번째 이미지, 해당 코드에서 n=5

std_cor = [] # 기준점 좌표
seg_cor_1 = [] # 첫 프레임의 segment point 좌표
ROI_cor = [] # 물 웅덩이 roi 좌표들
car_result = [1.5] # 자동차 환산비율 평균 (임의)
obj_result = [1.5] # 탐지 물체 환산비율 평균 (임의)


YOLO

In [10]:
'''
ROI 3: YOLO로 자동차 탐지 후 바운딩박스 생성

변수
unique_boxes: 바운딩 박스 좌표를 중복 저장하지 않기 위해 사용하는 set
class_id: 자동차의 class_id = 6
corners: YOLO 로 탐지한 bounding box의, 네 꼭짓점의 좌표

동작 과정
1. YOLO 로 첫번째 이미지 프레임에 있는 자동차 detect
2. 탐지 결과를 file_path 와 result_image_path 에 저장
3. 탐지 결과를 읽어들인 후, 조건에 맞는 객체 (자동차이며 신뢰도가 0.4 이상) 의 네 꼭짓점의 좌표 출력
4. 탐지 결과 이미지 보여줌
'''

# YOLO detection 관련 경로 설정
print(os.getcwd())
command = [
    './Algorithm/flooding/Scripts/python.exe', './YOLOv6-main/tools/infer.py',
    '--weights', './YOLOv6-main/runs/1000epochs_exp31_best_ckpt.pt',
    '--source', f'./example/' + location + '/4.jpg',
    '--yaml', './yolo/dataset.yaml',
    '--device', '0',
    '--save-txt', 
    '--save-dir', f'./test_result/' + location + '/'
]

subprocess.run(command)
print("==============")

file_path = f'./test_result/{location}/labels/{s}.txt'
result_image_path = f'./test_result/{location}/{s}.jpg'
result = []

with open(file_path, 'r') as file:
    for line in file:
        elements = line.strip().split()
        int_elements = list(map(float, elements))
        result.append(int_elements)

# 중복된 바운딩 박스 좌표를 저장하지 않도록 set 사용
unique_boxes = set()

cropped_car_images = []

# 바운딩 박스의 좌표 계산 및 출력
img_height, img_width = img_1.shape[:2]
for line in result:
    class_id = line[0]
    confidence = line[5]
    if class_id == 6 and confidence >= 0.4:
        # YOLO 형식: class, x_center, y_center, width, height, confidence
        x_center, y_center, width, height = line[1:5]

        # 이미지 좌표계로 변환
        x_center *= img_width
        y_center *= img_height
        width *= img_width
        height *= img_height

        # 바운딩 박스 좌표 (x_min, y_min, x_max, y_max)
        x_min = int(x_center - width / 2)
        y_min = int(y_center - height / 2)
        x_max = int(x_center + width / 2)
        y_max = int(y_center + height / 2)

        # 네 꼭짓점 좌표
        corners = ((x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max))

        # 중복되지 않은 바운딩 박스 좌표만 추가
        if corners not in unique_boxes:
            unique_boxes.add(corners)
            # 좌표 출력
            print(f"Car detected with bounding box corners: {corners}")

            # 바운딩 박스 영역을 이미지에서 크롭
            cropped_img = img_1[y_min:y_max, x_min:x_max]
            
            # 크롭된 이미지 저장
            cropped_car_images.append(cropped_img)

# 탐지된 결과를 보여주기
detected_img = cv2.imread(f'test_result/{location}/{s}.jpg', cv2.IMREAD_COLOR)
cv2.imshow('Detected Cars', detected_img)
cv2.waitKey(0)  # 키 입력을 기다림
cv2.destroyAllWindows()

d:\heejae\Flooding
Car detected with bounding box corners: ((959, 169), (986, 169), (986, 196), (959, 196))
Car detected with bounding box corners: ((511, 622), (714, 622), (714, 824), (511, 824))
Car detected with bounding box corners: ((838, 291), (875, 291), (875, 341), (838, 341))


YOLO를 통해 탐지된 결과 하나씩 보여줌

In [11]:
# 크롭된 자동차 이미지를 하나씩 보여주기
for idx, cropped_img in enumerate(cropped_car_images):
    # 이미지가 제대로 크롭되었는지 확인 후 표시
    if cropped_img is not None and cropped_img.size > 0:
        # 창 이름에 인덱스를 붙여 구분
        window_name = f'Cropped Car {idx + 1}'
        cv2.imshow(window_name, cropped_img)
        
        # 키 입력을 기다림 (0이면 무한 대기)
        cv2.waitKey(0)
        
        # 창 닫기
        cv2.destroyWindow(window_name)
    else:
        print(f"Warning: Cropped Car {idx + 1} is None or empty")

# 모든 창 닫기
cv2.destroyAllWindows()

# Module 4

In [5]:
# Device 설정 (GPU가 있으면 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((125, 125)),
    transforms.ToTensor()
])

# VGG16 모델 초기화
model = models.vgg16()

# 마지막 레이어를 3개의 클래스를 예측하도록 수정
model.classifier[6] = torch.nn.Linear(4096, 3)

# 모델을 GPU 또는 CPU로 전송
model = model.to(device)

checkpoint_path = r'D:\heejae\Flooding\module4\checkpoint\vgg16_final_weights.pth'  # 저장된 체크포인트 파일 경로
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint)  # 체크포인트에서 가중치 로드
    model.eval()
else:
    print("Checkpoint 파일을 찾을 수 없습니다.")
    exit()

# 이미지 배열 전처리
def preprocess_image(cropped_img):
    img_pil = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
    img_tensor = preprocess(img_pil)
    img_tensor = img_tensor.unsqueeze(0).to(device)  # 배치 차원 추가 및 디바이스로 이동
    return img_tensor

# YOLO를 통해 탐지된 자동차 이미지에 대해 VGG16 모델 예측 
def predict_vgg16(model, cropped_car_images):
    class_names = ['0', '1', '2']  # 예시 클래스 이름 (필요 시 수정)
    for idx, cropped_img in enumerate(cropped_car_images):
        if cropped_img is not None and cropped_img.size > 0:
            img_tensor = preprocess_image(cropped_img)
            with torch.no_grad():
                output = model(img_tensor)
                _, predicted = torch.max(output, 1)
                predicted_class = class_names[predicted.item()]
                print(f"Predicted class for Cropped Car {idx + 1}: {predicted_class}")
        else:
            print(f"Warning: Cropped Car {idx + 1} is None or empty")

# YOLO로 탐지된 자동차 이미지 리스트(cropped_car_images)를 VGGNet에 입력하여 예측 실행
predict_vgg16(model, cropped_car_images)

C:\Users\T719\AppData\Local\Temp\ipykernel_21072\2060372243.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device

Predicted class for Cropped Car 1: 2
